# HyTML - HTML templator / generator for Hy

In [1]:
(import hy sys)
(print "Hy version: " hy.__version__)
(print "Python" sys.version)

Hy version:  0.12.1
Python 3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


In [2]:
(require [hytml.macros [*]])
; to render html rather than display as a string
(import IPython)
(defmacro html+ [&rest code]
  `(IPython.display.HTML (html* ~@code)))

In [3]:
#@(html
    (head (title "Page title"))
    (body (div "Page content" :class container)))

'<html><head><title>Page title</title></head><body><div class="container">Page content</div></body></html>'

In [4]:
(html*
  (html
    (head (title "Page title"))
    (body (div "Page content" :class container))))

'<html><head><title>Page title</title></head><body><div class="container">Page content</div></body></html>'

In [5]:
(html+ "Content is " (b king) !)

In [6]:
(html* (apply sum [[1 2 3 4]]))

'<apply>sum<[1 2 3 4]></[1 2 3 4]></apply>'

In [7]:
; TODO variables, for loops and if clauses
(html* 
  (table (thead
    (tr 
      (for [cell ["col1" "col2" "col3"]]
        (td cell))))))

'<table><thead><tr><for><cell><col1>col2col3</col1></cell><td>cell</td></for></tr></thead></table>'

## The [MIT](http://choosealicense.com/licenses/mit/) License

Copyright (c) 2017 Marko Manninen